# Data Preparation

In [55]:
# !pip install -U -q PyDrive
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials
# # Authenticate and create the PyDrive client.
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

In [56]:
# from google.colab import drive
# drive.mount('/content/drive/')
# import os
# os.chdir('drive/My Drive/Colab Notebooks/Manchester Collaborate/[Joint-NLP][n2c2data]')

In [57]:
# install
!pip install pytorch-pretrained-bert pytorch-nlp

# BERT imports
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertModel
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
#% matplotlib inline

# specify GPU device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://mirrors.bfsu.edu.cn/pypi/web/simple


'NVIDIA GeForce RTX 3090'

In [58]:
import json
train_data = json.load(open("Data/train_ins_gs.json"))
dev_data = json.load(open("Data/valid_ins_gs.json"))
test_data = json.load(open("Data/test_ins_gs.json"))

In [59]:
train_seq = train_data['sentence']
dev_seq = dev_data['sentence']
test_seq = test_data['sentence']
train_slot = train_data['gt']
dev_slot = dev_data['gt']
test_slot = test_data['gt']

#POS Tagging

In [60]:
#obtain POS tag
from keras_preprocessing.sequence import pad_sequences
import numpy as np
import nltk
from nltk import pos_tag
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/mist/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [61]:
def tag_pos(train_sentences, dev_sentences, test_sentences):
      pos_tagged_sent = []

      pos_tagged_sent_train = []
      for sent in train_sentences:
          pos_tagged_sent.extend(pos_tag(sent))
          pos_tagged_sent_train.append(pos_tag(sent))

      pos_tagged_sent_dev = []
      for sent in dev_sentences:
          pos_tagged_sent_dev.append(pos_tag(sent))

      pos_tagged_sent_test = []
      for sent in test_sentences:
          pos_tagged_sent_test.append(pos_tag(sent))

      tags = list(set([i[1] for i in pos_tagged_sent]))

      pos_input_dim = len(tags)
      char_to_int = dict((c, i) for i, c in enumerate(tags))
      int_to_char = dict((i, c) for i, c in enumerate(tags))

      train_pos_encoded =[]
      for i in range(len(pos_tagged_sent_train)):
          temp = [pos[1] for pos in pos_tagged_sent_train[i]]
          train_pos_encoded.append(temp)
      
      dev_pos_encoded =[]
      for i in range(len(pos_tagged_sent_dev)):
          temp = [pos[1] for pos in pos_tagged_sent_dev[i]]
          dev_pos_encoded.append(temp)

      test_pos_encoded =[]
      for i in range(len(pos_tagged_sent_test)):
          temp = [pos[1] for pos in pos_tagged_sent_test[i]]
          test_pos_encoded.append(temp)

      return train_pos_encoded, dev_pos_encoded, test_pos_encoded

In [62]:
train_seq_pos, dev_seq_pos, test_seq_pos = tag_pos(train_seq, dev_seq, test_seq)

In [63]:
train_intent = ['no intent' for _ in range(0, len(train_seq))]
train_intent = tuple(train_intent)
dev_intent = ['no intent' for _ in range(0, len(dev_seq))]
dev_intent = tuple(dev_intent)
test_intent = ['no intent' for _ in range(0, len(test_seq))]
test_intent = tuple(test_intent)

In [64]:
print("#"*100)
print("Training data: "+str(len(train_data['sentence'])))
print("Development data: "+str(len(dev_data['sentence'])))
print("Testing data: "+str(len(test_data['sentence'])))
print("#"*100)
print("Data example:")
print("Sequence: ")
print(test_seq[5])
print()
print("Slots: ")
print(test_slot[5])
print()
print("POS: ")
print(test_seq_pos[5])
print("#"*100)

####################################################################################################
Training data: 41496
Development data: 4538
Testing data: 30614
####################################################################################################
Data example:
Sequence: 
['She', 'had', 'fevers', ',', 'weakness', ',', 'and', 'diarrhea', '.']

Slots: 
['O', 'O', 'B-ADE', 'O', 'B-ADE', 'O', 'O', 'B-ADE', 'O']

POS: 
['PRP', 'VBD', 'NNS', ',', 'NN', ',', 'CC', 'NN', '.']
####################################################################################################


In [65]:
slot_all=set()
pos_all=set()
intent_all = set()

for train_sequ, train_int in zip(train_slot, train_seq_pos):
  for i in train_sequ:
    slot_all.add(i)
  for i in train_int:
    pos_all.add(i)
    
for train_int in train_intent:
  intent_all.add(train_int)

#temp=['<PAD>']
#temp.extend(list(slot_all))
slot_all.add('<PAD>')
slot_all.add('<UNK>')
slot_all.add('[CLS]')
slot_all.add('[SEP]')
# slot_all.add('[CLS]')
pos_all.add('<PAD>')
pos_all.add('<UNK>')
pos_all.add('[CLS]')
pos_all.add('[SEP]')
# pos_all=list(pos_all)
intent_all.add('<UNK>')
intent_all=list(intent_all)


slot_num=len(slot_all)
pos_num=len(pos_all)
intent_num=len(intent_all)

print("#"*85)
print("Total slots: " + str(slot_all))
print("Total pos: " + str(pos_all))


print("#"*85)

print("#"*85)
print("Total slots type: " + str(slot_num))
print("Total pos type: " + str(pos_num))
print("Total intents type: " + str(intent_num))

print("#"*85)

print(list(slot_all)[-5:])
print(list(pos_all)[-5:])


print("#"*85)

#####################################################################################
Total slots: {'<UNK>', 'O', 'B-Strength', '<PAD>', 'B-Frequency', 'I-Drug', 'I-ADE', 'I-Strength', 'B-ADE', 'I-Dosage', 'I-Frequency', 'I-Duration', 'B-Drug', 'I-Reason', 'I-Form', 'B-Route', 'B-Dosage', '[CLS]', 'B-Form', 'B-Reason', 'B-Duration', 'I-Route', '[SEP]'}
Total pos: {'JJ', '#', '<UNK>', '.', '<PAD>', ')', 'DT', 'RP', 'RBR', 'PDT', "''", 'VBP', '(', 'SYM', 'NN', 'TO', 'RB', 'PRP', 'VBG', 'EX', 'NNS', 'WP', 'FW', 'JJS', 'IN', 'NNPS', 'PRP$', 'VBD', ':', 'MD', 'WP$', 'VBZ', 'LS', 'RBS', '[CLS]', 'WDT', 'WRB', 'CD', 'CC', 'POS', 'NNP', ',', '$', 'UH', 'VB', 'JJR', 'VBN', '[SEP]'}
#####################################################################################
#####################################################################################
Total slots type: 23
Total pos type: 48
Total intents type: 2
####################################################################################

In [66]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
lEnc_slot = LabelEncoder()
lEnc_pos = LabelEncoder()
lEnc_intent = LabelEncoder()

slots=np.array(list(slot_all))
pos=np.array(list(pos_all))
intents=np.array(list(intent_all))
lEnc_slot.fit(slots)
lEnc_pos.fit(pos)
lEnc_intent.fit(intents)

integer_mapping_slot = {l: i for i, l in enumerate(lEnc_slot.classes_)}
integer_mapping_pos = {l: i for i, l in enumerate(lEnc_pos.classes_)}
integer_mapping_intent = {l: i for i, l in enumerate(lEnc_intent.classes_)}
print(integer_mapping_slot)
print(integer_mapping_pos)
print(integer_mapping_intent)

train_slot_transformed=[]
dev_slot_transformed=[]
test_slot_transformed=[]
for seq_tr in train_slot:
  label=[integer_mapping_slot[i] if i in integer_mapping_slot else integer_mapping_slot['<UNK>'] for i in seq_tr]
  train_slot_transformed.append(label)

for seq_de in dev_slot:
  label=[integer_mapping_slot[i] if i in integer_mapping_slot else integer_mapping_slot['<UNK>'] for i in seq_de]
  dev_slot_transformed.append(label)

for seq_te in test_slot:
  label=[integer_mapping_slot[i] if i in integer_mapping_slot else integer_mapping_slot['<UNK>'] for i in seq_te]
  test_slot_transformed.append(label) 
    

train_pos_transformed=[]
dev_pos_transformed=[]
test_pos_transformed=[]
for seq_tr in train_seq_pos:
  label=[integer_mapping_pos[i] if i in integer_mapping_pos else integer_mapping_pos['<UNK>'] for i in seq_tr]
  train_pos_transformed.append(label)

for seq_de in dev_seq_pos:
  label=[integer_mapping_pos[i] if i in integer_mapping_pos else integer_mapping_pos['<UNK>'] for i in seq_de]
  dev_pos_transformed.append(label)

for seq_te in test_seq_pos:
  label=[integer_mapping_pos[i] if i in integer_mapping_pos else integer_mapping_pos['<UNK>'] for i in seq_te]
  test_pos_transformed.append(label) 

train_intent_transformed=lEnc_intent.transform(train_intent)
dev_intent_transformed=[integer_mapping_intent[i] if i in integer_mapping_intent else integer_mapping_intent['<UNK>'] for i in dev_intent]
test_intent_transformed=[integer_mapping_intent[i] if i in integer_mapping_intent else integer_mapping_intent['<UNK>'] for i in test_intent]
 
# train_pos_transformed=lEnc_intent.transform(train_seq_pos)
# dev_pos_transformed=[integer_mapping_intent[i] if i in integer_mapping_intent else integer_mapping_intent['<UNK>'] for i in dev_seq_pos]
# test_pos_transformed=[integer_mapping_intent[i] if i in integer_mapping_intent else integer_mapping_intent['<UNK>'] for i in test_seq_pos]
 
pad_slot_id=integer_mapping_slot['<PAD>']

{'<PAD>': 0, '<UNK>': 1, 'B-ADE': 2, 'B-Dosage': 3, 'B-Drug': 4, 'B-Duration': 5, 'B-Form': 6, 'B-Frequency': 7, 'B-Reason': 8, 'B-Route': 9, 'B-Strength': 10, 'I-ADE': 11, 'I-Dosage': 12, 'I-Drug': 13, 'I-Duration': 14, 'I-Form': 15, 'I-Frequency': 16, 'I-Reason': 17, 'I-Route': 18, 'I-Strength': 19, 'O': 20, '[CLS]': 21, '[SEP]': 22}
{'#': 0, '$': 1, "''": 2, '(': 3, ')': 4, ',': 5, '.': 6, ':': 7, '<PAD>': 8, '<UNK>': 9, 'CC': 10, 'CD': 11, 'DT': 12, 'EX': 13, 'FW': 14, 'IN': 15, 'JJ': 16, 'JJR': 17, 'JJS': 18, 'LS': 19, 'MD': 20, 'NN': 21, 'NNP': 22, 'NNPS': 23, 'NNS': 24, 'PDT': 25, 'POS': 26, 'PRP': 27, 'PRP$': 28, 'RB': 29, 'RBR': 30, 'RBS': 31, 'RP': 32, 'SYM': 33, 'TO': 34, 'UH': 35, 'VB': 36, 'VBD': 37, 'VBG': 38, 'VBN': 39, 'VBP': 40, 'VBZ': 41, 'WDT': 42, 'WP': 43, 'WP$': 44, 'WRB': 45, '[CLS]': 46, '[SEP]': 47}
{'<UNK>': 0, 'no intent': 1}


In [67]:
# Tokenize with BERT tokenizer
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# Tokenize with ClinicalBERT tokenizer
from transformers import AutoTokenizer, AutoModel
# tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT", do_lower_case=True)
# Tokenize with ClinicalBERT tokenizer

tokenizer = AutoTokenizer.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract", do_lower_case=True)

train_mid = [["[CLS]"] + query[:50] + ["[SEP]"] for query in train_seq]
dev_mid = [["[CLS]"] + query[:50] + ["[SEP]"] for query in dev_seq]
test_mid = [["[CLS]"] + query[:50] + ["[SEP]"] for query in test_seq]
train_mid=[[tokenizer.tokenize(w)[0] for w in seq] for seq in train_mid]
dev_mid=[[tokenizer.tokenize(w)[0] for w in seq] for seq in dev_mid]
test_mid=[[tokenizer.tokenize(w)[0] for w in seq] for seq in test_mid]

train_slot_mid = [[integer_mapping_slot['[CLS]']] + list(query) + [integer_mapping_slot['[SEP]']] for query in train_slot_transformed]
dev_slot_mid = [[integer_mapping_slot['[CLS]']] + list(query) + [integer_mapping_slot['[SEP]']] for query in dev_slot_transformed]
test_slot_mid = [[integer_mapping_slot['[CLS]']] + list(query) + [integer_mapping_slot['[SEP]']] for query in test_slot_transformed]

print(train_mid[0])
print(dev_mid[0])
print(test_mid[0])
print(train_slot_mid[0])
print(dev_slot_mid[0])
print(test_slot_mid[0])

['[CLS]', 'admission', 'date', ':', '[', '*', '*', '217', '-', '7', '-', '5', '*', '*', ']', 'discharge', 'date', ':', '[', '*', '*', '217', '-', '7', '-', '7', '*', '*', ']', 'service', ':', 'surgery', 'allergies', ':', 'amoxicillin', '/', 'penicillin', '/', 'coum', '/', 'oxy', '/', 'meg', 'acetate', '/', 'rem', '/', 'rit', 'attending', 'name', '(', '[SEP]']
['[CLS]', '[SEP]']
['[CLS]', 'admission', 'date', ':', '[', '*', '*', '210', '-', '7', '-', '21', '*', '*', ']', 'discharge', 'date', ':', '[SEP]']
[21, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 4, 20, 4, 20, 4, 20, 4, 20, 4, 4, 20, 4, 20, 4, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 22]
[21, 22]
[21, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20,

In [68]:
MAX_LEN = 52
# Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
# Pad our input tokens
train_input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in train_mid],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
dev_input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in dev_mid],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
test_input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in test_mid],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

train_slot_ids = pad_sequences(train_slot_mid,
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post", value=pad_slot_id)

dev_slot_ids = pad_sequences(dev_slot_mid,
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post", value=pad_slot_id)
test_slot_ids = pad_sequences(test_slot_mid,
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post", value=pad_slot_id)

In [69]:
# Create attention masks
train_attention_masks = []
dev_attention_masks = []
test_attention_masks = []
# Create a mask of 1s for each token followed by 0s for padding
for seq in train_input_ids:
  seq_mask = [float(i>0) for i in seq]
  train_attention_masks.append(seq_mask)
for seq in dev_input_ids:
  seq_mask = [float(i>0) for i in seq]
  dev_attention_masks.append(seq_mask)
for seq in test_input_ids:
  seq_mask = [float(i>0) for i in seq]
  test_attention_masks.append(seq_mask)


In [70]:
# Convert all of our data into torch tensors, the required datatype for our model

train_inputs = torch.tensor(train_input_ids)
dev_inputs = torch.tensor(dev_input_ids)
test_inputs = torch.tensor(test_input_ids)

train_labels = torch.tensor(train_intent_transformed)
dev_labels = torch.tensor(dev_intent_transformed)
test_labels = torch.tensor(test_intent_transformed)

train_slots = torch.tensor(train_slot_ids)
dev_slots = torch.tensor(dev_slot_ids)
test_slots = torch.tensor(test_slot_ids)

train_masks = torch.tensor(train_attention_masks)
dev_masks = torch.tensor(dev_attention_masks)
test_masks = torch.tensor(test_attention_masks)

In [71]:
# Select a batch size for training. 
batch_size = 32

# Create an iterator of our data with torch DataLoader 
train_data = TensorDataset(train_inputs, train_masks, train_labels, train_slots)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

dev_data = TensorDataset(dev_inputs, dev_masks, dev_labels, dev_slots)
dev_sampler = SequentialSampler(dev_data)
dev_dataloader = DataLoader(dev_data, sampler=dev_sampler, batch_size=batch_size)

test_data = TensorDataset(test_inputs, test_masks, test_labels, test_slots)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

train_sampler_ = SequentialSampler(train_data)
train_dataloader_ = DataLoader(train_data, sampler=train_sampler_, batch_size=batch_size)


In [72]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import accuracy_score
from torch.nn import CrossEntropyLoss, MSELoss

In [73]:
from transformers import AutoModelForSequenceClassification
class BertForSlotFilling(nn.Module):
  def __init__(self, num_labels, ignore_index=None):
    super(BertForSlotFilling, self).__init__()
    self.num_labels=num_labels
    self.ignore_index=ignore_index
    # self.bert=BertModel.from_pretrained("bert-base-uncased")
    # microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract
    
    self.bert=AutoModel.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract", output_hidden_states=True)
    self.dropout = nn.Dropout(0.1)
    self.classifier = nn.Linear(self.bert.config.hidden_size, num_labels)
    self._init_weights()


  def _init_weights(self):
    """ Initialize the weights """
    print("Initializing linear weight")
    self.classifier.weight.data.normal_(mean=0.0, std=self.bert.config.initializer_range)
  
  def forward(self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None):
    
    outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )
    sequence_output = outputs[0]
    sequence_output = self.dropout(sequence_output)

    logits = self.classifier(sequence_output)
    outputs = (logits,) + (outputs[0],)

    if labels is not None:
       loss_fct = nn.CrossEntropyLoss(ignore_index=self.ignore_index)
       if attention_mask is not None:
          active_loss = attention_mask.view(-1) == 1
          active_logits = logits.view(-1, self.num_labels)[active_loss]
          active_labels = labels.view(-1)[active_loss]
          loss = loss_fct(active_logits, active_labels)
       else:
          loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
       outputs = (loss,) + outputs

    return outputs  # (loss), logits, (hidden_states), (attentions)

In [74]:
# Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top. 
model_slot=BertForSlotFilling(slot_num, pad_slot_id)
model_slot.cuda()

Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Initializing linear weight


BertForSlotFilling(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine

In [75]:
# BERT fine-tuning parameters
param_optimizer = list(model_slot.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

optimizer = BertAdam(optimizer_grouped_parameters,
                     lr=2e-5,
                     warmup=.1)

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)
  
# Store our loss and accuracy for plotting
train_loss_set = []
# Number of training epochs 
epochs = 10

t_total value of -1 results in schedule not being applied


In [76]:
# BERT training loop
for _ in trange(epochs, desc="Epoch"):  
  
  ## TRAINING
  
  # Set our model to training mode
  model_slot.train()  
  # Tracking variables
  tr_loss = 0
  nb_tr_examples, nb_tr_steps = 0, 0
  # Train the data for one epoch
  for step, batch in enumerate(train_dataloader):
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, _, b_slot_labels = batch
    # Clear out the gradients (by default they accumulate)
    optimizer.zero_grad()
    # Forward pass
    (loss, _, _, ) = model_slot(input_ids=b_input_ids, attention_mask=b_input_mask, labels=b_slot_labels)

    train_loss_set.append(loss.item())    
    # Backward pass
    loss.backward()
    # Update parameters and take a step using the computed gradient
    optimizer.step()
    # Update tracking variables
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1
  print("Train loss: {}".format(tr_loss/nb_tr_steps))

Epoch:  10%|█         | 1/10 [02:44<24:40, 164.49s/it]

Train loss: 0.09037472169886508


Epoch:  20%|██        | 2/10 [05:29<21:57, 164.71s/it]

Train loss: 0.04684897442411132


Epoch:  30%|███       | 3/10 [08:14<19:15, 165.06s/it]

...

Train loss: 0.015768882738400387


Epoch:  90%|█████████ | 9/10 [24:59<02:47, 167.95s/it]

Train loss: 0.014529930192853953


Epoch: 100%|██████████| 10/10 [27:46<00:00, 166.66s/it]

Train loss: 0.013013862431845135


In [77]:
def process_mask(mask_in_batch):
  for i,mask in enumerate(mask_in_batch):
    sep_index=int(mask.sum()-1)
    b_input_mask[i][0]=0
    b_input_mask[i][sep_index]=0
  return mask_in_batch

In [78]:
# Put model in evaluation mode
model_slot.eval()

train_slot_embedding=[]
for batch in train_dataloader_:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, _, b_slot_labels = batch

  # Telling the model not to compute or store gradients, saving memory and speeding up validation
  with torch.no_grad():
    # Forward pass, calculate logit predictions
    (_, _, hidden_layer, ) = model_slot(input_ids=b_input_ids, attention_mask=b_input_mask, labels=b_slot_labels) 
    dim=hidden_layer.size(-1)
    b_input_mask=process_mask(b_input_mask)
    b_input_mask=b_input_mask.unsqueeze(-1).repeat(1,1,dim)
    embed=hidden_layer*b_input_mask
    train_slot_embedding.extend(np.array(embed[:,1:-1,:].cpu()))  


dev_slot_embedding=[]
for batch in dev_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, _, b_slot_labels = batch
  # Telling the model not to compute or store gradients, saving memory and speeding up validation
  with torch.no_grad():
    # Forward pass, calculate logit predictions
    (_, _, hidden_layer, ) = model_slot(input_ids=b_input_ids, attention_mask=b_input_mask, labels=b_slot_labels) 
    dim=hidden_layer.size(-1)
    b_input_mask=process_mask(b_input_mask)
    b_input_mask=b_input_mask.unsqueeze(-1).repeat(1,1,dim)
    embed=hidden_layer*b_input_mask
    dev_slot_embedding.extend(np.array(embed[:,1:-1,:].cpu()))  

test_slot_embedding=[]
for batch in test_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, _, b_slot_labels = batch
  # Telling the model not to compute or store gradients, saving memory and speeding up validation
  with torch.no_grad():
    # Forward pass, calculate logit predictions
    (_, _, hidden_layer, ) = model_slot(input_ids=b_input_ids, attention_mask=b_input_mask, labels=b_slot_labels) 
    dim=hidden_layer.size(-1)
    b_input_mask=process_mask(b_input_mask)
    b_input_mask=b_input_mask.unsqueeze(-1).repeat(1,1,dim)
    embed=hidden_layer*b_input_mask
    test_slot_embedding.extend(np.array(embed[:,1:-1,:].cpu()))   

In [79]:
train_slot_embedding=np.array(train_slot_embedding)
dev_slot_embedding=np.array(dev_slot_embedding)
test_slot_embedding=np.array(test_slot_embedding)
print(train_slot_embedding.shape)
print(dev_slot_embedding.shape)
print(test_slot_embedding.shape)

(41496, 50, 768)
(4538, 50, 768)
(30614, 50, 768)


In [80]:
import h5py

with h5py.File("Data/PUBMEDBERT/train_pubmedbert_ins_gs.h5", 'w') as f:
  f["data"] = np.array(train_slot_embedding)

with h5py.File("Data/PUBMEDBERT/dev_pubmedbert_ins_gs.h5", 'w') as f:
  f["data"] = np.array(dev_slot_embedding)

with h5py.File("Data/PUBMEDBERT/test_pubmedbert_ins_gs.h5", 'w') as f:
  f["data"] = np.array(test_slot_embedding)

In [81]:
###############1129 end here 